In [1]:
# define knowledge base 
knowledge_base = {
    "student": { "id": -2, "type": "academic (primary -> high school)"},
    "engineer student": { "id": -1, "type": "academic (college)"},
    "industrial management engineer": { "id": 0, "type": "profession"},
    "industrial management": { "id": 1, "type": "field"},
    "software": { "id": 2, "type": "stuff"},
    "software engineer": { "id": 3, "type": "profession"},
    "software engineer enthusiast": { "id": 4, "type": "hobby"},
    "emines": {"id": 5, "type": "institution"},
}

def query_method(s: str):
        return knowledge_base.get(s, None)

In [2]:
text = ("Five years before i was a student now i am an engineer student at EMINES."
        "school of industrial management by the end of this year i will graduate as."
        "An industrial management engineer, I am a software engineer enthusiast nothing to say student and student.")

In [3]:
from skillner.core import Pipeline, Document, Sentence

from skillner.text_loaders import StrTextLoader
from skillner.matchers import SlidingWindowMatcher

In [4]:
# define extraction pipeline
doc = Document()

extraction_pipeline = Pipeline()

extraction_pipeline.add_node(
    StrTextLoader(text),
    name='loader'
)

extraction_pipeline.add_node(
    SlidingWindowMatcher(
        query_method,
        max_window_size=4,
        pre_filter=lambda word: word.lower()
    ),
    name='matcher'
)

extraction_pipeline.run(doc)

In [5]:
# print matching result as table
for idx, sentence in enumerate(doc, start=1):

    print(f"Sentence {idx}")
    print("-" * len("Sentence 0"))

    print(f"{'position':10} | {'count candidates':20} | {'largest candidate':30}")
    print("-" * (10 + 20 + 30 + 3 * 2))

    for span in sentence.li_spans:
        position = f"({span.start}, {span.stop})"
        count_spans = len(span.li_candidates)
        max_candidate = max(
            span.li_candidates, key=lambda candidate: len(candidate)
        )

        str_max_candidate = " ".join(sentence[max_candidate.window])
        print(f"{position:10} | {count_spans:20} | {str_max_candidate:30} ")

    print("=" * (10 + 20 + 30 + 3 * 2))

Sentence 1
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(6, 7)     |                    1 | student                        
(11, 13)   |                    1 | engineer student               
(12, 13)   |                    1 | student                        
(14, 15)   |                    1 | EMINES                         
Sentence 2
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(2, 4)     |                    1 | industrial management          
Sentence 3
----------
position   | count candidates     | largest candidate             
------------------------------------------------------------------
(1, 4)     |                    2 | industrial management engineer 
(7, 10)    |                    3 | software engineer enthusiast   
(13, 14)   |                    1 | student             